```
@作者: 阿凯爱玩机器人
@QQ: 244561792
@微信: xingshunkai
@邮箱: xingshunkai@qq.com
@网址: deepsenserobot.com
@B站: "阿凯爱玩机器人"
```

## 导入依赖

In [1]:
import numpy as np
import open3d as o3d
# 自定义库
from kyle_robot_toolbox.open3d import *

pybullet build time: Jun  3 2022 02:05:55


## 载入点云

In [2]:
scene_pcd = o3d.io.read_point_cloud(f"./data/example/box_pose/pcd_rect_roi.pcd")
# 点云可视化
o3d.visualization.draw_geometries([scene_pcd])

## 平面拟合

In [3]:
# 距离阈值 单位m
# 距离平面多少的点被认为是在这个平面上的点
distance_threshold = 0.005
# 平面拟合
plane_model, inliers = scene_pcd.segment_plane(distance_threshold=distance_threshold,
                                         ransac_n=3,
                                         num_iterations=500)
# 打印拟合平面的表达式
[a, b, c, d] = plane_model
print(f"拟合桌面平面的表达式: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

# 工作平面上的点云
ws_panel_cloud = scene_pcd.select_by_index(inliers)
ws_panel_cloud.paint_uniform_color([1.0, 0, 0])


draw_geometry([ws_panel_cloud])

拟合桌面平面的表达式: 0.05x + 0.01y + 1.00z + -0.26 = 0


In [4]:
# 保存平面拟合的结果
# 保存平面信息
panel_model = np.float32([a, b, c, d])
np.savetxt("./data/example/box_pose/desktop_panel_model.txt", panel_model, delimiter=",", fmt="%.4f")

##  移除工作台平面后的点云

In [5]:
# 工作台平面上的点云
scene_pcd_on_panel = scene_pcd.select_by_index(inliers)

# 将工作平面上的点云移除
scene_pcd_rm_panel = scene_pcd.select_by_index(inliers, invert=True)
# 可视化
draw_geometry([scene_pcd_rm_panel])

## 移除平面后方的点

在点云边缘存在噪声，有些噪声在拟合得到的工作台平面后方。 
把这些点移除

In [6]:
points_3d = np.asarray(scene_pcd_rm_panel.points)
# 分别获取X坐标, Y坐标, Z坐标的列表
x_list = points_3d[:, 0]
y_list = points_3d[:, 1]
z_list = points_3d[:, 2]

In [7]:
# 获取平面内侧的点云
value = a*x_list + b*y_list + c*z_list + d
pcd_close_panel_index = np.argwhere(value > 0)

scene_pcd_close_panel = scene_pcd_rm_panel.select_by_index(pcd_close_panel_index, invert=True)
draw_geometry([scene_pcd_close_panel])

![](./image/移除平面下方的点云.png)

## 拟合盒子上表面

In [8]:
# 距离阈值 单位m
# 距离平面多少的点被认为是在这个平面上的点
distance_threshold = 0.01
# 平面拟合
box_plane_model, inliers = scene_pcd_close_panel.segment_plane(distance_threshold=distance_threshold,
                                         ransac_n=3,
                                         num_iterations=500)
# 打印拟合平面的表达式
[a, b, c, d] = box_plane_model
print(f"拟合盒子平面的表达式: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

# 工作平面上的点云
box_panel_pcd = scene_pcd_close_panel.select_by_index(inliers)
box_panel_pcd.paint_uniform_color([0, 1.0, 0])

draw_geometry([box_panel_pcd, scene_pcd_on_panel])

拟合盒子平面的表达式: 0.08x + 0.01y + 1.00z + -0.23 = 0


In [9]:
# 保存平面拟合的结果
# 保存平面信息
panel_model = np.float32([a, b, c, d])
np.savetxt("./data/example/box_pose/box_panel_model.txt", box_plane_model, delimiter=",", fmt="%.4f")

In [10]:
# 保存盒子表面的点云
# 保存PCD点云 二进制格式
o3d.io.write_point_cloud("./data/example/box_pose/box_panel_pcd.pcd", box_panel_pcd)

True